In [38]:
using DrWatson
@quickactivate "HEMI" 
using HEMI 
using DataFrames, Chain
using CSV, StringEncodings
using Plots

## Métricas de evaluación histórica

In [2]:
## Directorios de resultados 
config_savepath = datadir("results", "corr-combination", "Esc-F")
tray_dir = datadir(config_savepath, "tray_infl")

# CountryStructure con datos hasta período de evaluación 
gtdata_eval = gtdata[Date(2020, 12)]

UniformCountryStructure{2, Float32, Float32} con 2 bases
|─> VarCPIBase{Float32, Float32}: 120 períodos × 218 gastos básicos Jan-2001-Dec-2010
|─> VarCPIBase{Float32, Float32}: 120 períodos × 279 gastos básicos Jan-2011-Dec-2020


In [5]:
df_results = collect_results(config_savepath)
df_results

┌ Info: Scanning folder c:\Users\RRCP\Documents\HEMI\HEMI\data\results\corr-combination\Esc-F for result files.
└ @ DrWatson C:\Users\RRCP\.julia\packages\DrWatson\VwP8P\src\result_collection.jl:88
┌ Info: Added 7 entries.
└ @ DrWatson C:\Users\RRCP\.julia\packages\DrWatson\VwP8P\src\result_collection.jl:118


,gt_b00_mse_bias,traindate,inflfn
,Float32?,Date?,Inflati…?
1,0.0384297,2020-12-01,"CombinationFunction{3, Vector{Float32}}(EnsembleFunction{3}((InflationCoreMai{Float32, Float64, MaiG{Vector{Float64}}}(-200.0f0:0.01f0:200.0f0, MaiG{Vector{Float64}}(3, [0.0, 0.323195, 0.77172, 1.0])), InflationCoreMai{Float32, Float64, MaiFP{Vector{Float64}}}(-200.0f0:0.01f0:200.0f0, MaiFP{Vector{Float64}}(5, [0.0, 0.335389, 0.65647, 0.781121, 0.860586, 1.0])), InflationCoreMai{Float32, Float64, MaiF{Vector{Float64}}}(-200.0f0:0.01f0:200.0f0, MaiF{Vector{Float64}}(5, [0.0, 0.318405, 0.656481, 0.777264, 0.871024, 1.0])))), Float32[0.00581682, 0.434901, 0.55931], ""MAI óptima de correlación 2018"")"
2,3.15955,2020-12-01,"InflationDynamicExclusion(0.468323, 4.97451)"
3,0.107584,2020-12-01,"InflationFixedExclusionCPI{2}(([35, 30, 190, 36, 37, 40, 31, 104, 162, 32, 33, 159], [29, 116, 31, 46, 39, 40, 186, 30, 35, 185, 197, 34, 48, 184, 41, 47, 37, 22, 25, 229, 38, 32, 274, 3, 45, 44, 33, 237, 19, 10, 24, 275, 115, 15, 59, 42, 61, 43, 113, 49, 27, 71, 23, 268, 9, 36, 236, 78, 20, 213, 273, 26]))"
4,5.03112,2020-12-01,"InflationTrimmedMeanEq(55.9051, 92.1777)"
5,44.0879,2020-12-01,"InflationTrimmedMeanWeighted(46.4432, 98.5461)"
6,4.26757,2020-12-01,InflationPercentileEq(0.772522)
7,33.4896,2020-12-01,InflationPercentileWeighted(0.809557)


### Métricas de los 4 períodos

In [6]:
select(df_results, :measure, r"corr")

,measure,gt_b00_corr,gt_t0010_corr,gt_b10_corr
,String?,Float32?,Float32?,Float32?
1,MAI óptima de correlación 2018,0.971953,0.980825,0.876439
2,"Inflación de exclusión dinámica (0.47, 4.97)",0.940207,0.972252,0.722675
3,"Exclusión fija de gastos básicos IPC (12, 52)",0.946636,0.969423,0.613
4,"Media Truncada Equiponderada (55.91, 92.18)",0.9722,0.978092,0.872161
5,"Media Truncada Ponderada (46.44, 98.55)",0.942932,0.972701,0.734026
6,Percentil equiponderado 77.25,0.968672,0.978261,0.871594
7,Percentil ponderado 80.96,0.932154,0.974553,0.756394


### Métricas de la combinación óptima CORR

In [7]:
optcorr2022 = wload(datadir(config_savepath, "optcorr2022", "optcorr2022.jld2"), "optcorr2022")
components_df = components(optcorr2022)

,measure,weights
,String,Float32
1,Percentil ponderado 80.96,3.93042e-5
2,"Inflación de exclusión dinámica (0.47, 4.97)",0.00194286
3,"Media Truncada Ponderada (46.44, 98.55)",0.0
4,Percentil equiponderado 77.25,0.19322
5,MAI óptima de correlación 2018,0.336925
6,"Media Truncada Equiponderada (55.91, 92.18)",0.467873
7,"Exclusión fija de gastos básicos IPC (12, 52)",0.0


In [8]:
# DataFrame de combinación 
combine_df = @chain df_results begin 
    select(:measure, :corr, :me, :inflfn, 
        :path => ByRow(p -> joinpath(tray_dir, basename(p))) => :tray_path)
    sort(:corr)
end
combine_df = leftjoin(combine_df, components_df, on = :measure)
combine_df

## Obtener las trayectorias de los archivos guardados en el directorio tray_infl 
# Genera un arreglo de 3 dimensiones de trayectorias (T, n, K)
tray_infl = mapreduce(hcat, combine_df.tray_path) do path
    load(path, "tray_infl")
end

functions = combine_df.inflfn
components_mask = [!(fn isa InflationFixedExclusionCPI) for fn in functions];

In [9]:
# Obtener el parámetro
resamplefn = df_results[1, :resamplefn]
trendfn = df_results[1, :trendfn]
paramfn = df_results[1, :paramfn]
param = InflationParameter(paramfn, resamplefn, trendfn)
tray_infl_pob = param(gtdata_eval);

In [10]:
# Pesos de la combinación óptima
corr_weights = combine_df.weights

EVAL_PERIODS = [GT_EVAL_B00, GT_EVAL_T0010, GT_EVAL_B10, CompletePeriod()]

comb_metrics = mapreduce(merge, EVAL_PERIODS) do period
    # Ventana de evaluación
    eval_window = eval_periods(gtdata_eval, period)
    # Trayectorias combinadas
    tray_infl_opt = sum(tray_infl[eval_window, :, :] .* corr_weights', dims=2)
    metrics = eval_metrics(tray_infl_opt, tray_infl_pob[eval_window], prefix = period_tag(period))
end;


In [11]:
# Obtener un DataFrame de métricas de la combinación lineal CORR
comb_metrics_df = DataFrame(comb_metrics)
comb_metrics_df[!, :measure] .= measure_name(optcorr2022)
select(comb_metrics_df, :measure, r"corr")

,measure,corr,gt_b00_corr,gt_b10_corr,gt_t0010_corr
,String,Float32,Float32,Float32,Float32
1,Subyacente óptima CORR 2022,0.987285,0.972561,0.876094,0.979512


In [15]:
historic_df = [
    select(df_results, :measure, r"corr");
    select(comb_metrics_df, :measure, r"corr")
]

,measure,gt_b00_corr,gt_t0010_corr,gt_b10_corr
,String?,Float32?,Float32?,Float32?
1,MAI óptima de correlación 2018,0.971953,0.980825,0.876439
2,"Inflación de exclusión dinámica (0.47, 4.97)",0.940207,0.972252,0.722675
3,"Exclusión fija de gastos básicos IPC (12, 52)",0.946636,0.969423,0.613
4,"Media Truncada Equiponderada (55.91, 92.18)",0.9722,0.978092,0.872161
5,"Media Truncada Ponderada (46.44, 98.55)",0.942932,0.972701,0.734026
6,Percentil equiponderado 77.25,0.968672,0.978261,0.871594
7,Percentil ponderado 80.96,0.932154,0.974553,0.756394
8,Subyacente óptima CORR 2022,0.972561,0.979512,0.876094


## Métricas en período de optimización (Dic-2001 - Dic-2018)

In [17]:
## Directorios de resultados 
config_savepath = datadir("results", "corr-combination", "Esc-F18")
tray_dir = datadir(config_savepath, "tray_infl")

# CountryStructure con datos hasta período de evaluación 
gtdata_eval = gtdata[Date(2018, 12)]

# DataFrame con resultados de evaluación hasta dic-18
df_results = collect_results(config_savepath)
df_results

┌ Info: Scanning folder c:\Users\RRCP\Documents\HEMI\HEMI\data\results\corr-combination\Esc-F18 for result files.
└ @ DrWatson C:\Users\RRCP\.julia\packages\DrWatson\VwP8P\src\result_collection.jl:88


┌ Info: Added 7 entries.
└ @ DrWatson C:\Users\RRCP\.julia\packages\DrWatson\VwP8P\src\result_collection.jl:118


,gt_b00_mse_bias,traindate,inflfn
,Float32?,Date?,Inflati…?
1,0.0399365,2018-12-01,"CombinationFunction{3, Vector{Float32}}(EnsembleFunction{3}((InflationCoreMai{Float32, Float64, MaiG{Vector{Float64}}}(-200.0f0:0.01f0:200.0f0, MaiG{Vector{Float64}}(3, [0.0, 0.323195, 0.77172, 1.0])), InflationCoreMai{Float32, Float64, MaiFP{Vector{Float64}}}(-200.0f0:0.01f0:200.0f0, MaiFP{Vector{Float64}}(5, [0.0, 0.335389, 0.65647, 0.781121, 0.860586, 1.0])), InflationCoreMai{Float32, Float64, MaiF{Vector{Float64}}}(-200.0f0:0.01f0:200.0f0, MaiF{Vector{Float64}}(5, [0.0, 0.318405, 0.656481, 0.777264, 0.871024, 1.0])))), Float32[0.00581682, 0.434901, 0.55931], ""MAI óptima de correlación 2018"")"
2,3.15955,2018-12-01,"InflationDynamicExclusion(0.468323, 4.97451)"
3,0.107584,2018-12-01,"InflationFixedExclusionCPI{2}(([35, 30, 190, 36, 37, 40, 31, 104, 162, 32, 33, 159], [29, 116, 31, 46, 39, 40, 186, 30, 35, 185, 197, 34, 48, 184, 41, 47, 37, 22, 25, 229, 38, 32, 274, 3, 45, 44, 33, 237, 19, 10, 24, 275, 115, 15, 59, 42, 61, 43, 113, 49, 27, 71, 23, 268, 9, 36, 236, 78, 20, 213, 273, 26]))"
4,5.03112,2018-12-01,"InflationTrimmedMeanEq(55.9051, 92.1777)"
5,44.0879,2018-12-01,"InflationTrimmedMeanWeighted(46.4432, 98.5461)"
6,4.26757,2018-12-01,InflationPercentileEq(0.772522)
7,33.4896,2018-12-01,InflationPercentileWeighted(0.809557)


In [28]:
# Se utiliza solamente :corr porque se desea únicament el período completo de
# estos datos (hasta dic-18) para obtener las métricas de optimización
select(df_results, :measure, :corr)

,measure,corr
,String?,Float32?
1,MAI óptima de correlación 2018,0.985625
2,"Inflación de exclusión dinámica (0.47, 4.97)",0.976798
3,"Exclusión fija de gastos básicos IPC (12, 52)",0.977486
4,"Media Truncada Equiponderada (55.91, 92.18)",0.985724
5,"Media Truncada Ponderada (46.44, 98.55)",0.977656
6,Percentil equiponderado 77.25,0.984079
7,Percentil ponderado 80.96,0.973637


### Métricas de la combinación óptima CORR en período de optimización

In [21]:
# DataFrame de combinación 
combine_df = @chain df_results begin 
    select(:measure, :corr, :me, :inflfn, 
        :path => ByRow(p -> joinpath(tray_dir, basename(p))) => :tray_path)
    sort(:corr)
end
combine_df = leftjoin(combine_df, components_df, on = :measure)
combine_df

## Obtener las trayectorias de los archivos guardados en el directorio tray_infl 
# Genera un arreglo de 3 dimensiones de trayectorias (T, n, K)
tray_infl = mapreduce(hcat, combine_df.tray_path) do path
    load(path, "tray_infl")
end

functions = combine_df.inflfn
components_mask = [!(fn isa InflationFixedExclusionCPI) for fn in functions];

In [24]:
# Obtener el parámetro
resamplefn = df_results[1, :resamplefn]
trendfn = df_results[1, :trendfn]
paramfn = df_results[1, :paramfn]
param = InflationParameter(paramfn, resamplefn, trendfn)
tray_infl_pob = param(gtdata_eval);

In [27]:
# Pesos de la combinación óptima
corr_weights = combine_df.weights

EVAL_PERIODS = [EvalPeriod(Date(2001, 12), Date(2018, 12), "opt18")]

comb_metrics = mapreduce(merge, EVAL_PERIODS) do period
    # Ventana de evaluación
    eval_window = eval_periods(gtdata_eval, period)
    # Trayectorias combinadas
    tray_infl_opt = sum(tray_infl[eval_window, :, :] .* corr_weights', dims=2)
    metrics = eval_metrics(tray_infl_opt, tray_infl_pob[eval_window], prefix = period_tag(period))
end

# Obtener un DataFrame de métricas de la combinación lineal CORR
comb_metrics_df = DataFrame(comb_metrics)
comb_metrics_df[!, :measure] .= measure_name(optcorr2022)
select(comb_metrics_df, :measure, r"corr")

,measure,opt18_corr
,String,Float32
1,Subyacente óptima CORR 2022,0.986085


In [31]:
# Combinar todas las métricas en período de optimización 
optim_period_df = [
    select(df_results, :measure, :corr => :opt18_corr); 
    select(comb_metrics_df, :measure, r"corr")
]

,measure,opt18_corr
,String?,Float32?
1,MAI óptima de correlación 2018,0.985625
2,"Inflación de exclusión dinámica (0.47, 4.97)",0.976798
3,"Exclusión fija de gastos básicos IPC (12, 52)",0.977486
4,"Media Truncada Equiponderada (55.91, 92.18)",0.985724
5,"Media Truncada Ponderada (46.44, 98.55)",0.977656
6,Percentil equiponderado 77.25,0.984079
7,Percentil ponderado 80.96,0.973637
8,Subyacente óptima CORR 2022,0.986085


## Combinar las métricas en un DataFrame de salida

In [43]:
final_df = leftjoin(historic_df, optim_period_df, on=:measure)

,measure,gt_b00_corr,gt_t0010_corr,gt_b10_corr
,String?,Float32?,Float32?,Float32?
1,MAI óptima de correlación 2018,0.971953,0.980825,0.876439
2,"Inflación de exclusión dinámica (0.47, 4.97)",0.940207,0.972252,0.722675
3,"Exclusión fija de gastos básicos IPC (12, 52)",0.946636,0.969423,0.613
4,"Media Truncada Equiponderada (55.91, 92.18)",0.9722,0.978092,0.872161
5,"Media Truncada Ponderada (46.44, 98.55)",0.942932,0.972701,0.734026
6,Percentil equiponderado 77.25,0.968672,0.978261,0.871594
7,Percentil ponderado 80.96,0.932154,0.974553,0.756394
8,Subyacente óptima CORR 2022,0.972561,0.979512,0.876094


In [39]:
# Guardar los resultados en un archivo CSV
function save_csv(file::AbstractString, df::DataFrame)
    encoding = enc"ISO-8859-1"
    @info "Saving file with $encoding" file
    open(file, encoding, "w") do io 
        CSV.write(io, df)
    end
end

metrics_savepath = mkpath(datadir("updates", "metrics"))
save_csv(
    joinpath(metrics_savepath, "optcorr2022_metrics.csv"),
    final_df
)

┌ Info: Saving file with ISO-8859-1
│   file = c:\Users\RRCP\Documents\HEMI\HEMI\data\updates\metrics\optcorr2022_metrics.csv
└ @ Main c:\Users\RRCP\Documents\HEMI\HEMI\notebooks\historic-combination-metrics\corr-combination-metrics.ipynb:4


StringEncoder{UTF-8, ISO-8859-1}(IOStream(<file c:\Users\RRCP\Documents\HEMI\HEMI\data\updates\metrics\optcorr2022_metrics.csv>))